## Step 1: Load packages, dataset and check the data we working with.

* Is there anything wrong with the data?
* Are there any anormalities with the data?
* Do I need to fix or remove any of the data including null values?



In [9]:
#importing necesaary packages

import numpy as np
import pandas as pd
import matplotlib as plt

In [14]:
df = pd.read_csv('/Users/mzwakhebesho/Documents/DPY401T Assignment/Regression Problem/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


The data is usable.

We can gather the following:
* The first row in the data file defines the column headers
* The headers are understandable enough to what each column represents.
* Each row represents an entry for an individual: 6 input data points and one target, which tells us the cost amount of the insurance.